In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import numpy as np
import torch
import matplotlib.pyplot as plt

from Inference import Inferencer

import IPython.display as ipd

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
inferencer = Inferencer(
    hp_path= 'Hyper_Parameters.yaml',
    checkpoint_path= '/data/results/DiffWave/Exp1/Checkpoint/S_80000.pt',
    batch_size= 1
    )

2022-09-26 18:44:09,315 (Inference:47) INFO: Checkpoint loaded at 80000 steps.


In [3]:
import librosa
from meldataset import mel_spectrogram

audio, _ = librosa.load('./inference_wav/p226_003.wav', sr= inferencer.hp.Sound.Sample_Rate)
audio = librosa.effects.trim(audio, top_db=60, frame_length= 512, hop_length= 256)[0]
audio = librosa.util.normalize(audio) * 0.95
audio = audio[:audio.shape[0] - (audio.shape[0] % inferencer.hp.Sound.Frame_Shift)]

features = mel_spectrogram(
    y= torch.from_numpy(audio).float().unsqueeze(0),
    n_fft= inferencer.hp.Sound.N_FFT,
    num_mels= inferencer.hp.Sound.Mel_Dim,
    sampling_rate= inferencer.hp.Sound.Sample_Rate,
    hop_size= inferencer.hp.Sound.Frame_Shift,
    win_size= inferencer.hp.Sound.Frame_Length,
    fmin= inferencer.hp.Sound.Mel_F_Min,
    fmax= inferencer.hp.Sound.Mel_F_Max,
    center= False
    )

# features = features[:, :, 0:384]
feature_lengths = [features.size(2)]

In [7]:
audio = inferencer.Inference(
    features= features,
    feature_lengths= feature_lengths
    )[0]
ipd.display(ipd.Audio(audio, rate= inferencer.hp.Sound.Sample_Rate))

[Diffusion]: 100%|██████████████████████████████████████████████████████████████████| 1000/1000 [12:17<00:00,  1.36it/s]


In [8]:
audio = inferencer.Inference_DDIM(
    features= features,
    feature_lengths= feature_lengths,
    steps= 100
    )
ipd.display(ipd.Audio(audio, rate= inferencer.hp.Sound.Sample_Rate))

[Diffusion]: 100%|████████████████████████████████████████████████████████████████████| 100/100 [01:12<00:00,  1.38it/s]
